In [1]:
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
import itertools
import distance
import cPickle
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import time
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')
eng_stopwords = stopwords.words('english')
import xgboost as xgb
#from scipy.spatial import distance
import re
import sys
print sys.getfilesystemencoding()

UTF-8


In [2]:
#Reading 50000 Rows for Pre Production

In [3]:
%%time

data = pd.read_csv('/home/nlp/Downloads/q/train.csv',nrows=10)
data.columns


CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 11.2 ms


In [4]:
#data = pd.read_csv("/home/nlp/Downloads/Dataset/Quora/Feature_Output_10k.csv")

In [5]:
test = pd.read_csv("/home/nlp/Downloads/q/test.csv")

In [6]:
del data['id']
del data['qid1']
del data['qid2']

In [7]:
data.head(2)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [8]:
# Data Cleaning - Remove Punc,lower / Stop Words Not Removed

In [9]:
%%time
def cleaning(s):
    s = str(s)
    s = s.lower()
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W\s',' ',s)
    s = re.sub('\s+',' ',s)
    return s
data['question1'] = [cleaning(s) for s in data['question1']]
data['question2'] = [cleaning(s) for s in data['question2']]

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.64 ms


In [ ]:
%%time
import re, math
from collections import Counter

#WORD = re.compile(r'\w+')

#def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)
data['text1'] = [[]] * len(data)
data['text2'] = [[]] * len(data)
for i in range(len(data)):
    data['text1'][i] = text_to_vector(data['question1'][i])
    data['text2'][i] = text_to_vector(data['question2'][i])

In [15]:
# Feature Set 

In [10]:
%%time
#data['cosine'] = [[]] * len(data)
#for i in range(len(data)):
#    data['cosine'][i] = get_cosine(data['text1'][i], data['text2'][i])
    
data['levinstein'] = [[]] * len(data)
for i in range(len(data)):
    data['levinstein'][i] = distance.levenshtein(data['question1'][i], data['question2'][i])
    
data['jaccard'] = [[]] * len(data)
for i in range(len(data)):
    data['jaccard'][i] = jaccard(data['question1'][i],data['question2'][i])
    
data['sorensen'] = [[]] * len(data)
for i in range(len(data)):
    data['sorensen'][i] = distance.sorensen(data['question1'][i], data['question2'][i])
    
data['bleu'] = [[]] * len(data)
for i in range(len(data)):
    data['bleu'][i] = nltk.translate.bleu_score.sentence_bleu(data['question1'][i],data['question2'][i])
       

/home/nlp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nlp/anaconda2/lib/python2.7/site-packages/scipy/spatial/distance.py:436: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.bitwise_or(u != 0, v != 0)).sum())
/home/nlp/anaconda2/lib/python2.7/site-packages/scipy/spatial/distance.py:437: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  / np.double(np.bitwise_or(u != 0, v != 0).sum()))
/home/nlp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://

CPU times: user 2.2 s, sys: 12 ms, total: 2.21 s
Wall time: 2.19 s


In [10]:
%%time
data['len_q1'] = data.question1.apply(lambda x : len(str(x)))
data['len_q2'] = data.question2.apply(lambda x : len(str(x)))
data['diff_len'] = data.len_q1 - data.len_q2
data['len_char_q1'] = data.question1.apply(lambda x : len(''.join(set(str(x).replace('','')))))
data['len_char_q2'] = data.question2.apply(lambda x : len(''.join(set(str(x).replace('','')))))
data['len_word_q1'] = data.question1.apply(lambda x : len(str(x).split()))
data['len_word_q2'] = data.question2.apply(lambda x : len(str(x).split()))
data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 15 ms


In [11]:
%%time
data['fuzz_QRatio'] = data.apply(lambda x : fuzz.QRatio(str(x['question1']),str(x['question2'])),axis=1)
data['fuzz_WRatio'] = data.apply(lambda x : fuzz.WRatio(str(x['question1']),str(x['question2'])),axis=1)
data['fuzzy_PartialRatio'] = data.apply(lambda x : fuzz.partial_ratio(str(x['question1']),str(x['question2'])),axis=1)
data['fuzzy_Partial_set_Token_Ratio'] = data.apply(lambda x : fuzz.partial_token_set_ratio(str(x['question1']),str(x['question2'])),axis=1)
data['fuzzy_Partial_sort_Token_Ratio'] = data.apply(lambda x : fuzz.partial_token_sort_ratio(str(x['question1']),str(x['question2'])),axis=1)
data['fuzzy_Token_Set_Ratio'] = data.apply(lambda x :fuzz.token_set_ratio(str(x['question1']),str(x['question2'])),axis=1)
data['fuzzy_Token_Sort_Ratio'] = data.apply(lambda x :fuzz.token_sort_ratio(str(x['question1']),str(x['question2'])),axis=1)


CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 41.2 ms


In [12]:
def get_unigrams(que):
    return [word for word in word_tokenize(que.lower()) if word not in stop_words]

def get_common_unigrams(row):
    return len( set(row["unigrams_ques1"]).intersection(set(row["unigrams_ques2"])) )

def get_common_unigram_ratio(row):
    return float(row["unigrams_common_count"]) / max(len( set(row["unigrams_ques1"]).union(set(row["unigrams_ques2"])) ),1)

data["unigrams_ques1"] = data['question1'].apply(lambda x: get_unigrams(str(x)))
data["unigrams_ques2"] = data['question2'].apply(lambda x: get_unigrams(str(x)))
data["unigrams_common_count"] = data.apply(lambda row: get_common_unigrams(row),axis=1)
data["unigrams_common_ratio"] = data.apply(lambda row: get_common_unigram_ratio(row), axis=1)



/home/nlp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  from ipykernel import kernelapp as app


In [13]:
def get_bigrams(que):
    return [i for i in ngrams(que, 2)]

def get_common_bigrams(row):
    return len( set(row["bigrams_ques1"]).intersection(set(row["bigrams_ques2"])) )

def get_common_bigram_ratio(row):
    return float(row["bigrams_common_count"]) / max(len( set(row["bigrams_ques1"]).union(set(row["bigrams_ques2"])) ),1)

data["bigrams_ques1"] = data["unigrams_ques1"].apply(lambda x: get_bigrams(x))
data["bigrams_ques2"] = data["unigrams_ques2"].apply(lambda x: get_bigrams(x)) 
data["bigrams_common_count"] = data.apply(lambda row: get_common_bigrams(row),axis=1)
data["bigrams_common_ratio"] = data.apply(lambda row: get_common_bigram_ratio(row), axis=1)



In [14]:
#w2v
%time
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

model = Word2Vec.load_word2vec_format('/home/nlp/Documents/Vectors/GoogleNews-vectors-negative300.bin', binary=True)
data['wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 19.1 µs


/home/nlp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/home/nlp/anaconda2/lib/python2.7/site-packages/gensim-0.13.4.1-py2.7-linux-x86_64.egg/gensim/models/keyedvectors.py:353: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return word in self.vocab


In [16]:
#w2v norm
%time
def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


norm_model = Word2Vec.load_word2vec_format('/home/nlp/Documents/Vectors/GoogleNews-vectors-negative300.bin', binary=True)
norm_model.init_sims(replace=True)
data['norm_wmd'] = data.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11 µs


/home/nlp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [21]:
def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())




question1_vectors = np.zeros((data.shape[0], 300))
error_count = 0

for i, q in tqdm(enumerate(data.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((data.shape[0], 300))
for i, q in tqdm(enumerate(data.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

0it [00:00, ?it/s]


UnicodeDecodeError: 'utf8' codec can't decode byte 0x82 in position 16: invalid start byte

In [17]:
#data = pd.read_csv('/home/nlp/Downloads/Dataset/Quora/Final_Output.csv')


In [18]:
data.head(2).T

,0,1
question1,what is the step by step guide to invest in sh...,what is the story of kohinoor koh-i-noor diamond?
question2,what is the step by step guide to invest in sh...,what would happen if the indian government sto...
is_duplicate,0,0
levinstein,9,42
jaccard,1,1
sorensen,0,0.190476
bleu,0.779087,0.666788
len_q1,66,49
len_q2,57,86
diff_len,9,-37


In [18]:

def feature_extraction(row):
    que1 = str(row['question1'])
    que2 = str(row['question2'])
    out_list = []
    # get unigram features #
    unigrams_que1 = [word for word in que1.lower().split() if word not in eng_stopwords]
    unigrams_que2 = [word for word in que2.lower().split() if word not in eng_stopwords]
    common_unigrams_len = len(set(unigrams_que1).intersection(set(unigrams_que2)))
    common_unigrams_ratio = float(common_unigrams_len) / max(len(set(unigrams_que1).union(set(unigrams_que2))),1)
    out_list.extend([common_unigrams_len, common_unigrams_ratio])

    # get bigram features #
    bigrams_que1 = [i for i in ngrams(unigrams_que1, 2)]
    bigrams_que2 = [i for i in ngrams(unigrams_que2, 2)]
    common_bigrams_len = len(set(bigrams_que1).intersection(set(bigrams_que2)))
    common_bigrams_ratio = float(common_bigrams_len) / max(len(set(bigrams_que1).union(set(bigrams_que2))),1)
    out_list.extend([common_bigrams_len, common_bigrams_ratio])

    # get trigram features #
    trigrams_que1 = [i for i in ngrams(unigrams_que1, 3)]
    trigrams_que2 = [i for i in ngrams(unigrams_que2, 3)]
    common_trigrams_len = len(set(trigrams_que1).intersection(set(trigrams_que2)))
    common_trigrams_ratio = float(common_trigrams_len) / max(len(set(trigrams_que1).union(set(trigrams_que2))),1)
    out_list.extend([common_trigrams_len, common_trigrams_ratio])
    return out_list

In [19]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0):
        params = {}
        params['learning_rate'] =0.01
        params["objective"] = "binary:logistic"
        params['eval_metric'] = 'logloss'
        params["eta"] = 0.02
        params["subsample"] = 0.7
        params["min_child_weight"] = 1
        params["colsample_bytree"] = 0.7
        params["max_depth"] = 4
        params["silent"] = 1
        params["seed"] = seed_val
        num_rounds = 300 
        plst = list(params.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        if test_y is not None:
                xgtest = xgb.DMatrix(test_X, label=test_y)
                watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
                model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=10)
        else:
                xgtest = xgb.DMatrix(test_X)
                model = xgb.train(plst, xgtrain, num_rounds)
                
        pred_test_y = model.predict(xgtest)

        loss = 1
        if test_y is not None:
                loss = log_loss(test_y, pred_test_y)
                return pred_test_y, loss, model
        else:
            return pred_test_y, loss, model

In [20]:
train_X = np.vstack( np.array(data.apply(lambda row: feature_extraction(row), axis=1)) ) 
test_X = np.vstack( np.array(test.apply(lambda row: feature_extraction(row), axis=1)) )
train_y = np.array(data["is_duplicate"])
test_id = np.array(test["test_id"])

/home/nlp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/home/nlp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [21]:


train_X_dup = train_X[train_y==1]
train_X_non_dup = train_X[train_y==0]

train_X = np.vstack([train_X_non_dup, train_X_dup, train_X_non_dup, train_X_non_dup])
train_y = np.array([0]*train_X_non_dup.shape[0] + [1]*train_X_dup.shape[0] + [0]*train_X_non_dup.shape[0] + [0]*train_X_non_dup.shape[0])
del train_X_dup
del train_X_non_dup
print("Mean target rate : ",train_y.mean())



kf = KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
    dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    preds, lloss, model = runXGB(dev_X, dev_y, val_X, val_y)
    break



('Mean target rate : ', 0.16436353972893969)
[0]	train-logloss:0.68772	test-logloss:0.687707
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.639288	test-logloss:0.639006
[20]	train-logloss:0.599322	test-logloss:0.598802
[30]	train-logloss:0.565368	test-logloss:0.564723
[40]	train-logloss:0.537177	test-logloss:0.536397
[50]	train-logloss:0.513059	test-logloss:0.512215
[60]	train-logloss:0.493041	test-logloss:0.49212
[70]	train-logloss:0.475631	test-logloss:0.474659
[80]	train-logloss:0.459849	test-logloss:0.458912
[90]	train-logloss:0.446277	test-logloss:0.445403
[100]	train-logloss:0.43481	test-logloss:0.433952
[110]	train-logloss:0.424247	test-logloss:0.423463
[120]	train-logloss:0.414849	test-logloss:0.414203
[130]	train-logloss:0.406558	test-logloss:0.406038
[140]	train-logloss:0.400217	test-logloss:0.399721
[150]	train-logloss:0.394048	test-logloss:0.393669
[16

In [22]:
xgtest = xgb.DMatrix(test_X)
preds = model.predict(xgtest)

out_df = pd.DataFrame({"test_id":test_id, "is_duplicate":preds})
out_df.to_csv("/home/nlp/Desktop/xgb_starter.csv", index=False)



In [18]:
#word2vec

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [ ]:
#normWord2vec

In [ ]:
########Sentence to Vector#############

In [ ]:
def sent2vec(s):
    words = str(s).lower().decode('ISO-8859-1')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
question1_vectors = np.zeros((data.shape[0], 300))
question2_vectors = np.zeros((data.shape[0], 300))
error_count = 0

In [ ]:
for i, q in tqdm(enumerate(data.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

In [ ]:
for i, q in tqdm(enumerate(data.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

In [ ]:
cPickle.dump(question1_vectors, open('q1_w2v.pkl', 'wb'), -1)
cPickle.dump(question2_vectors, open('data/q2_w2v.pkl','wb'), -1)

In [ ]:
#data.to_csv('/home/nlp/Downloads/Dataset/Quora/quora_senfeatures.csv', index=False)